# Automatically download FESOM2 data from Zenodo based on catalog

## Imports and paths

In [1]:
import intake
import os
from pathlib import Path
from esmvfc_cattools import download_zenodo_files_for_entry
import logging

In [2]:
# parameters
data_path = Path("../esm_vfc_data/").resolve()
catalog_file = Path("../catalogs/FESOM2_PI_MESH.yaml")

In [3]:
os.environ["ESM_VFC_DATA_DIR"] = str(data_path)

## Open catalog and download all data

In [4]:
cat = intake.open_catalog(str(catalog_file))

In [5]:
list(cat)

['FESOM2_sample', 'MESH_NOD2D', 'MESH_ELEM2D', 'MESH_AUX3D']

In [6]:
for entry in [cat[name] for name in cat]:
    download_zenodo_files_for_entry(
        entry,
        force_download=True
    )

will download https://zenodo.org/api/files/21cd530d-24d2-46bb-94c5-dc0ba924b5a6/a_ice.fesom.1948.nc to /work/esm-vfc-catalogs/esm_vfc_data/FESOM2_PI_MESH/a_ice.fesom.1948.nc



will download https://zenodo.org/api/files/21cd530d-24d2-46bb-94c5-dc0ba924b5a6/Av.fesom.1948.nc to /work/esm-vfc-catalogs/esm_vfc_data/FESOM2_PI_MESH/Av.fesom.1948.nc



will download https://zenodo.org/api/files/21cd530d-24d2-46bb-94c5-dc0ba924b5a6/Kv.fesom.1948.nc to /work/esm-vfc-catalogs/esm_vfc_data/FESOM2_PI_MESH/Kv.fesom.1948.nc



will download https://zenodo.org/api/files/21cd530d-24d2-46bb-94c5-dc0ba924b5a6/m_ice.fesom.1948.nc to /work/esm-vfc-catalogs/esm_vfc_data/FESOM2_PI_MESH/m_ice.fesom.1948.nc



will download https://zenodo.org/api/files/21cd530d-24d2-46bb-94c5-dc0ba924b5a6/MLD1.fesom.1948.nc to /work/esm-vfc-catalogs/esm_vfc_data/FESOM2_PI_MESH/MLD1.fesom.1948.nc


...


will download https://zenodo.org/api/files/21cd530d-24d2-46bb-94c5-dc0ba924b5a6/vnod.fesom.1948.nc to /work/esm-vfc-catalogs/esm_vfc_data/FESOM2_PI_MESH/vnod.fesom.1948.nc



will download https://zenodo.org/api/files/21cd530d-24d2-46bb-94c5-dc0ba924b5a6/w.fesom.1948.nc to /work/esm-vfc-catalogs/esm_vfc_data/FESOM2_PI_MESH/w.fesom.1948.nc



will download https://zenodo.org/api/files/21cd530d-24d2-46bb-94c5-dc0ba924b5a6/pi.zip to /work/esm-vfc-catalogs/esm_vfc_data/FESOM2_PI_MESH/pi.zip



will download https://zenodo.org/api/files/21cd530d-24d2-46bb-94c5-dc0ba924b5a6/pi.zip to /work/esm-vfc-catalogs/esm_vfc_data/FESOM2_PI_MESH/pi.zip



will download https://zenodo.org/api/files/21cd530d-24d2-46bb-94c5-dc0ba924b5a6/pi.zip to /work/esm-vfc-catalogs/esm_vfc_data/FESOM2_PI_MESH/pi.zip


## Read catalog entries

In [8]:
print(cat["FESOM2_sample"].read())

<xarray.Dataset>
Dimensions:  (elem: 5839, nod2: 3140, nz: 48, nz1: 47, time: 12)
Coordinates:
  * time     (time) float64 2.678e+06 5.097e+06 ... 2.886e+07 3.154e+07
Dimensions without coordinates: elem, nod2, nz, nz1
Data variables:
    Av       (time, elem, nz) float32 0.005085066 0.099402376 ... 0.0 0.0
    Kv       (time, nod2, nz) float32 nan nan nan nan nan ... 0.0 0.0 0.0 0.0
    MLD1     (time, nod2) float32 -75.19909 -77.151375 ... -33.665993 -32.839382
    a_ice    (time, nod2) float32 0.98289865 0.9805132 0.9803011 ... 0.0 0.0 0.0
    m_ice    (time, nod2) float32 1.1300349 1.1106514 1.1073328 ... 0.0 0.0 0.0
    salt     (time, nod2, nz1) float32 nan nan nan nan nan ... 0.0 0.0 0.0 0.0
    ssh      (time, nod2) float32 -0.5813815 -0.5792128 ... -1.5763197 -1.586672
    sst      (time, nod2) float32 nan nan nan ... -1.517087 -1.5979521 -1.535592
    temp     (time, nod2, nz1) float32 nan nan nan nan nan ... 0.0 0.0 0.0 0.0
    u        (time, elem, nz1) float32 nan nan nan 

In [9]:
cat["MESH_AUX3D"].read()

,topo
0,-672
1,-534
2,-621
3,-744
4,-594
...,...
3135,-1200
3136,-850
3137,-560
3138,-121


In [10]:
print(cat["MESH_NOD2D"].read())

      node_number         x        y  flag
0               1  267.4665  84.5252     0
1               2  270.9702  84.3700     0
2               3  268.3207  84.0503     0
3               4  263.8669  84.5900     0
4               5  264.3323  84.2140     0
...           ...       ...      ...   ...
3135         3136  139.7089 -79.0173     0
3136         3137  144.1223 -79.2533     0
3137         3138  148.0000 -80.3240     1
3138         3139  135.7037 -79.9329     1
3139         3140  141.5839 -80.1193     1

[3140 rows x 4 columns]


In [11]:
print(cat["MESH_ELEM2D"].read())

      first_elem  second_elem  third_elem
0              1           12           2
1              2           12          10
2              2           10           9
3              3            1           2
4              3            5           1
...          ...          ...         ...
5834        3138         3137        3132
5835        3138         3131        3133
5836        3139         3136        3140
5837        3139         3135        3134
5838        3140         3137        3138

[5839 rows x 3 columns]
